In [1]:
import pandas as pd
from collections import Counter
import re
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

comp_color = {
    '魔坦刺': '#858561',
    '扶桑法刺': '#9e71ad',
    '扶桑群雄刺': '#968795',
    '扶桑群雄法刺': '#6603fc',
    '扶桑刺': '#6603fc',
    '魔种': '#a35956',
    '扶桑男刺': '#bfae91',
    '扶桑法': '#8000ff',

    '七射': '#14ffa5',
    '攻辅射': '#42d7f5',
    '弟弟射': '#fceb30',
    '攻辅蛋': '#ffac1c',

    '九战': '#ff0000',
    '稷下战': '#ff5e5e',
    '轻战守约': '#ff8e5e',
    '魏战': '#ff0084',

    '蜀国': '#00cc18',

    '法奶': '#0093f5',
    }

comp_to_brief = {
    '扶桑法刺': '扶桑刺',
    '扶桑群雄刺': '扶桑刺',
    '扶桑群雄法刺': '扶桑刺',
    '扶桑男刺': '扶桑刺',
    '魔种(无天赋)' :'魔种',
    '魔种(有天赋)' :'魔种',
    '封神攻辅射': '攻辅射',
    '吴国攻辅射': '攻辅射',
    '稷下攻辅射': '攻辅射'
}

rank_to_score = [12, 9, 7, 5, 3, 2, 1, 0]

class Config:
    use_brief_comp = True

In [2]:
df = pd.read_excel('result.xlsx')
pd.set_option('display.max_columns', None)
df = df.dropna()

if Config.use_brief_comp:
    df['详细阵容'] = df['阵容']
    df['阵容'] = df['详细阵容'].apply(lambda x: comp_to_brief.get(x, x))
display(df.head(3))
display(df.tail(3))

,排名,俱乐部,选手,ban位,阵容,金钱,输出,胜,负,局数,分组,周数,游戏版本,详细阵容
0,1,TES,阿虾,稷,魔坦刺,146.0,161.0,20.0,9.0,1.0,低分组,3,21-0,魔坦刺
1,2,RNGM,俊仔,稷,扶桑刺,80.0,168.0,20.0,9.0,1.0,低分组,3,21-0,扶桑法刺
2,3,VG,婷婷,卫,扶桑刺,71.0,115.0,17.0,11.0,1.0,低分组,3,21-0,扶桑群雄刺


,排名,俱乐部,选手,ban位,阵容,金钱,输出,胜,负,局数,分组,周数,游戏版本,详细阵容
157,6,WB.TS,仟谕,吴,稷下战,77.0,144.0,16.0,8.0,5.0,低分组,4,21-0,稷下战
158,7,杭州LGD大鹅,阿霖,稷,扶桑刺,77.0,78.0,13.0,11.0,5.0,低分组,4,21-0,扶桑群雄刺
159,8,上海EDG.M,墨翟,魏,七射,52.0,31.0,7.0,14.0,5.0,低分组,4,21-0,七射


In [3]:
def get_comp_statistics(df):
    def get_most_ban(x):
        c = Counter(x['ban位'])
        return [(i, c[i] / len(x)) for i, count in c.most_common()]

    groups = df.groupby('阵容')
    mean_rank = groups['排名'].mean().sort_values()
    var_rank = groups['排名'].var()
    sum_occur = groups['排名'].count()
    most_ban = groups.apply(get_most_ban)
    res = pd.concat([mean_rank, var_rank, sum_occur, most_ban], axis=1)
    res.columns = ('平均排名', '排名方差', '出场次数', '禁用统计')
    res = res.fillna('-')
    return res

def get_ban_statistics(df):
    def get_most_comp(x):
        c = Counter(x['阵容'])
        return [(i, c[i] / len(x)) for i, count in c.most_common()]
    groups = df.groupby('ban位')
    mean_rank = groups['排名'].mean().sort_values()
    var_rank = groups['排名'].var()
    sum_occur = groups['排名'].count()
    most_comp = groups.apply(get_most_comp)
    res = pd.concat([mean_rank, var_rank, sum_occur, most_comp], axis=1)
    res.columns = ('平均排名', '排名方差', '出场次数', '阵容统计')
    res = res.fillna('-')
    return res

def get_club_statistics(df):
    groups = df.groupby('俱乐部')
    mean_rank = groups['排名'].mean().sort_values()
    var_rank = groups['排名'].var()
    sum_occur = groups['排名'].count()
    res = pd.concat([mean_rank, var_rank, sum_occur], axis=1)
    res.columns = ('平均排名', '排名方差', '出场次数')
    res = res.fillna('-')
    return res

def get_top1_comp_statistics(df):
    groups = df.groupby('阵容')['排名']
    occur_cnt = groups.count()
    top1_cnt = groups.apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / occur_cnt
    res = pd.concat([top1_rate, top1_cnt, occur_cnt], axis=1)
    res.columns = ('登顶率', '登顶次数', '出场次数')
    res = res.sort_values(['登顶率', '登顶次数'], ascending=False)
    return res

def get_top3_comp_statistics(df):
    groups = df.groupby('阵容')['排名']
    occur_cnt = groups.count()
    top3_cnt = groups.apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / occur_cnt
    res = pd.concat([top3_rate, top3_cnt, occur_cnt], axis=1)
    res.columns = ('前三率', '前三次数', '出场次数')
    res = res.sort_values(['前三率', '前三次数'], ascending=False)
    return res

display(get_comp_statistics(df))
display(get_ban_statistics(df))
display(get_club_statistics(df))
display(get_top1_comp_statistics(df))
display(get_top3_comp_statistics(df))

,平均排名,排名方差,出场次数,禁用统计
攻辅蛋,2.000000,-,1,"[(蜀, 1.0)]"
弟弟射,3.428571,4.72527,14,"[(蜀, 1.0)]"
轻战守约,3.500000,12.5,2,"[(安, 0.5), (吴, 0.5)]"
扶桑法,4.000000,-,1,"[(卫, 1.0)]"
魏战,4.000000,-,1,"[(安, 1.0)]"
蜀国,4.285714,6.57143,7,"[(魏, 0.42857142857142855), (安, 0.2857142857142..."
扶桑刺,4.333333,4.95455,45,"[(卫, 0.4), (稷, 0.35555555555555557), (魏, 0.177..."
稷下战,4.406250,4.89415,32,"[(安, 0.59375), (魏, 0.1875), (蜀, 0.125), (吴, 0...."
攻辅射,4.444444,7.43791,18,"[(蜀, 0.3888888888888889), (卫, 0.33333333333333..."
魔坦刺,4.800000,7.7,5,"[(稷, 0.8), (神, 0.2)]"


,平均排名,排名方差,出场次数,阵容统计
蜀,3.875000,5.14516,32,"[(弟弟射, 0.4375), (攻辅射, 0.21875), (法奶, 0.125), (..."
安,3.888889,5.48718,27,"[(稷下战, 0.7037037037037037), (九战, 0.07407407407..."
卫,4.656250,4.87802,32,"[(扶桑刺, 0.5625), (法奶, 0.21875), (攻辅射, 0.1875), ..."
魏,4.666667,4.91954,30,"[(七射, 0.3), (扶桑刺, 0.26666666666666666), (稷下战, ..."
稷,4.714286,5.84127,28,"[(扶桑刺, 0.5714285714285714), (魔种, 0.25), (魔坦刺, ..."
吴,6.000000,0.666667,4,"[(稷下战, 0.5), (轻战守约, 0.25), (蜀国, 0.25)]"
神,6.000000,-,1,"[(魔坦刺, 1.0)]"
尧,6.666667,3.86667,6,"[(七射, 0.5), (扶桑刺, 0.16666666666666666), (攻辅射, ..."


,平均排名,排名方差,出场次数
西安WE,2.4,2.300000,5
NMG,3.0,2.000000,5
CW,3.2,8.700000,5
KSSC,3.8,5.700000,5
RNGM,4.0,6.500000,5
ACT,4.2,3.200000,5
KS.YTG,4.2,6.200000,5
LX蓝翔,4.2,6.700000,5
MQ,4.2,8.200000,5
DYG,4.4,7.300000,5


,登顶率,登顶次数,出场次数
阵容,,,
轻战守约,0.500000,1,2
攻辅射,0.222222,4,18
弟弟射,0.214286,3,14
魔坦刺,0.200000,1,5
蜀国,0.142857,1,7
稷下战,0.125000,4,32
扶桑刺,0.111111,5,45
法奶,0.090909,1,11
七射,0.000000,0,13


,前三率,前三次数,出场次数
阵容,,,
攻辅蛋,1.000000,1,1
弟弟射,0.571429,8,14
轻战守约,0.500000,1,2
攻辅射,0.444444,8,18
蜀国,0.428571,3,7
稷下战,0.406250,13,32
魔坦刺,0.400000,2,5
扶桑刺,0.377778,17,45
七射,0.307692,4,13


In [5]:
def plot_ban_to_comp(ban_stats, filepath=None):
    fig, axs = plt.subplots(1, len(ban_stats), figsize=(30, 4))
    for ax, ban in zip(axs, ban_stats.itertuples()):
        labels, ys = [m[0] for m in ban[4]], [m[1] for m in ban[4]]
        colors = [comp_color[re.sub(r'\([^)]*\)', '', c).strip()] for c in labels]
        explode = [0.015] * len(ys)
        def value(val):
            return f'{val:.2f}%'
        ax.pie(ys, labels=labels, colors=colors, explode=explode, autopct=value, shadow=False, pctdistance=0.6)
        ax.set_title(f'{ban[0]} (排名{ban[1]:.2f},出场{ban[3]})', fontsize=15, x=0.5, y=1.01)

    fig.tight_layout()
    if filepath:
        fig.savefig(filepath, dpi=350)
    else:
        plt.show(fig)
    plt.close(fig)

def plot_comp_data(comp_stats, filepath=None):
    fig, ax = plt.subplots(figsize=(6, 4))
    y = comp_stats['平均排名']
    x = comp_stats['出场次数']
    names = comp_stats.index
    ax.scatter(x, y)
    ax.set_title('阵容数据')
    ax.set_ylabel('平均排名')
    ax.set_xlabel('出场次数')
    ax.set_xlim(left=0, right=max(x) + 1)
    ax.set_ylim(bottom=1, top=8)
    ax.hlines(y=4.5, xmin=0, xmax=max(x) + 1, colors='orange', linestyles='--', lw=2, label='理论均值')
    ax.legend(loc="lower right")
    for i, txt in enumerate(names):
        ax.annotate(txt, (x[i] + 0.25, y[i]))
    fig.gca().invert_yaxis()
    if filepath:
        fig.savefig(filepath, dpi=350)
    else:
        plt.show(fig)
    plt.close(fig)

comp_stats = get_comp_statistics(df)
plot_comp_data(comp_stats, "data/comp_data.png")
# plot_comp_data(comp_stats, None)
ban_stats = get_ban_statistics(df)
plot_ban_to_comp(ban_stats, "data/ban_to_comp.png")
# plot_ban_to_comp(ban_stats, None)